In [1]:
import pandas as pd
import numpy as np

from collections import Counter

import re
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [2]:
#client_id = 'ee59408d22cb47209e1e8f735beae2d7'
#client_secret = '0bf85ab2cfe74bbf82cf8b565dc3a35b'

client_id = 'db6115dd9210469a8197c3be928129e3'
client_secret = '532606f782c3410e9591071f12f8838a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
auth_url = 'https://accounts.spotify.com/api/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}
auth_response = requests.post(auth_url, data=auth_data)
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
headers = {
    'Authorization': f'Bearer {access_token}',
}

In [4]:
def get_playlist_id_from_url(url):

    playlist_pattern = r'https://open\.spotify\.com/playlist/([a-zA-Z0-9]+)'    
    match = re.search(playlist_pattern, url)

    if match:
        playlist_id = match.group(1)
        return playlist_id
    else:
        return None


def get_playlist_tracks(playlist_id):

    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    return tracks


def get_track_mood(track_id):

    audio_features = sp.audio_features(track_id)

    if len(audio_features) > 0:
        return audio_features
    
    return None


def get_playlist_features(playlist_tracks):

    song_data = {
        'Track_Name': [],
        'Track_ID': [],
        'URL': [],
        'Artist(s)': [],
        'Album': [],
        'Popularity': [],
        'Duration': [],
        'Tempo': [],
        'Key': [],
        'Mode': [],
        'Danceability': [],
        'Energy': [],
        'Loudness': [],
        'Speechiness': [],
        'Acousticness': [],
        'Instrumentalness': [],
        'Liveliness': [],
        'Valence': [],
        'Time_Signature': []
    }
    
    artist_id = []

    for p in range(len(playlist_tracks)):
        f = get_track_mood(playlist_tracks[p]['track']['uri'])
        song_data['Track_Name'].append(playlist_tracks[p]['track']['name'])
        song_data['Track_ID'].append(playlist_tracks[p]['track']['uri'])
        song_data['URL'].append(playlist_tracks[p]['track']['external_urls']['spotify'])
        song_data['Artist(s)'].append([artist['name'] for artist in playlist_tracks[p]['track']['artists']])
        song_data['Album'].append(playlist_tracks[p]['track']['album']['name'])
        song_data['Popularity'].append(playlist_tracks[p]['track']['popularity'])
        song_data['Duration'].append(round(f[0]['duration_ms'] / 60000, 2))
        song_data['Tempo'].append(f[0]['tempo'])
        song_data['Key'].append(f[0]['key'])
        song_data['Mode'].append(f[0]['mode'])
        song_data['Danceability'].append(f[0]['danceability'])
        song_data['Energy'].append(f[0]['energy'])
        song_data['Loudness'].append(f[0]['loudness'])
        song_data['Speechiness'].append(f[0]['speechiness'])
        song_data['Acousticness'].append(f[0]['acousticness'])
        song_data['Instrumentalness'].append(f[0]['instrumentalness'])
        song_data['Liveliness'].append(f[0]['liveness'])
        song_data['Valence'].append(f[0]['valence'])
        song_data['Time_Signature'].append(f[0]['time_signature'])
        artist_id.extend([artist['id'] for artist in playlist_tracks[p]['track']['artists']])

    artists = {
        'Name': [],
        'ID': [],
        'Genres': [],
        'Followers': [],
        'Popularity': []
    }

    for a in set(artist_id):
        temp = sp.artist('spotify:artist:' + a)
        if temp['genres'] == []:
            continue
        artists['Name'].append(temp['name'])
        artists['ID'].append(temp['id'])
        artists['Genres'].append(temp['genres'])
        artists['Followers'].append(temp['followers']['total'])
        artists['Popularity'].append(temp['popularity'])
    
    return pd.DataFrame(song_data), pd.DataFrame(artists)

In [5]:
def search_songs_by_artist(artist_name):
    
    results = sp.search(q=f'artist:{artist_name}', type='track', limit=1)
    tracks = results['tracks']['items']

    if not tracks:
        return {}

    return tracks


def collab_network(artist_1, artist_2):
    
    collab = {
        "Artist 1": [],
        "Artist 2": [],
        "Popular Collab": [],
        "URL": [], 
        "Popularity": []
    }

    for a in artist_2:
        track = search_songs_by_artist(', '.join([artist_1, a]))
        if track != {}:
            if a in [z['name'] for z in track[0]['artists']] and artist_1 in [z['name'] for z in track[0]['artists']]:
                print(track[0]['name'], track[0]['external_urls']['spotify'], track[0]['popularity'])
                collab["Artist 1"].append(artist_1)
                collab["Artist 2"].append(a)
                collab["Popular Collab"].append(track[0]['name'])
                collab["URL"].append(track[0]['external_urls']['spotify'])
                collab["Popularity"].append(track[0]['popularity'])
    
    return pd.DataFrame(collab)

In [6]:
def get_top_artists_for_genre(genre, num_artists=5):
    results = sp.search(q=f'genre:{genre}', type='artist', limit=num_artists)
    artists = results['artists']['items']
    return artists

def get_top_tracks_for_artist(artist_id, num_tracks=3):
    tracks = sp.artist_top_tracks(artist_id)
    return tracks['tracks'][:num_tracks]

In [7]:
playlist_url_1 = 'https://open.spotify.com/playlist/1EPAHw66Ba1eZEVxffbHgz?si=5fffa0341f7e41eb'
playlist_url_2 = 'https://open.spotify.com/playlist/6GsfHNKqjeScWqakIBQjUM?si=ebf233a66c33468f'
#playlist_url = 'https://open.spotify.com/playlist/1EPAHw66Ba1eZEVxffbHgz?si=369dea96062d476a'
#playlist_url = 'https://open.spotify.com/playlist/3ErWnfK0liTwLCkfXty9KM'
playlist_id_1 = get_playlist_id_from_url(playlist_url_1)
playlist_tracks_1 = get_playlist_tracks(playlist_id_1)
playlist_id_2 = get_playlist_id_from_url(playlist_url_2)
playlist_tracks_2 = get_playlist_tracks(playlist_id_2)

In [9]:
playlist1, artist1 = get_playlist_features(playlist_tracks_1)
playlist2, artist2 = get_playlist_features(playlist_tracks_2)

In [10]:
playlist1['Playlist'] = 'Playlist 1'
playlist2['Playlist'] = 'Playlist 2'

In [11]:
playlists = pd.concat([playlist1, playlist2], ignore_index=True)
playlists.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Playlist
58,Save Your Tears (Remix) (with Ariana Grande) -...,spotify:track:1oFAF1hdPOickyHgbuRjyX,https://open.spotify.com/track/1oFAF1hdPOickyH...,"[The Weeknd, Ariana Grande]",After Hours (Deluxe),83,3.18,118.091,0,1,0.650,0.825,-4.645,0.0325,0.02150,0.000024,0.0936,0.593,4,Playlist 2
88,It Ain't Me (with Selena Gomez),spotify:track:3eR23VReFzcdmS7TYCrhCe,https://open.spotify.com/track/3eR23VReFzcdmS7...,"[Kygo, Selena Gomez]",It Ain't Me (with Selena Gomez),80,3.68,99.968,0,1,0.640,0.533,-6.596,0.0706,0.11900,0.000000,0.0864,0.515,4,Playlist 2
7,Slow Dancing in a Burning Room,spotify:track:2jdAk8ATWIL3dwT47XpRfu,https://open.spotify.com/track/2jdAk8ATWIL3dwT...,[John Mayer],Continuum,76,4.03,134.018,1,0,0.667,0.361,-8.690,0.0273,0.57500,0.005060,0.0805,0.290,5,Playlist 1
44,Hotel California - 2013 Remaster,spotify:track:40riOy7x9W7GXjyGp4pjAv,https://open.spotify.com/track/40riOy7x9W7GXjy...,[Eagles],Hotel California (2013 Remaster),85,6.52,147.125,2,1,0.579,0.508,-9.484,0.0270,0.00574,0.000494,0.0575,0.609,4,Playlist 1
50,When You're Gone,spotify:track:0U1W2LZVUX7qTm7dDpqxh6,https://open.spotify.com/track/0U1W2LZVUX7qTm7...,[Shawn Mendes],When You're Gone,78,2.87,147.148,5,1,0.603,0.691,-5.805,0.0372,0.02480,0.000000,0.5830,0.577,4,Playlist 2


In [12]:
all_artists = pd.concat([artist1, artist2], ignore_index=True)
which_playlist = []
for a in all_artists['Name']:
    if a in list(artist1['Name']) and a in list(artist2['Name']):
        which_playlist.append("Both")
    elif a in list(artist1['Name']):
        which_playlist.append("Playlist 1")
    else:
        which_playlist.append("Playlist 2")
all_artists['Playlist'] = which_playlist
all_artists = all_artists.loc[all_artists.astype(str).drop_duplicates().index]
all_artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist
70,Niall Horan,1Hsdzj7Dlq2I7tHP7501T4,[pop],9882217,74,Playlist 2
43,BANNERS,4qWnlmXWuGv2TtuxtIWlJX,[indie poptimism],630316,63,Playlist 2
37,Kygo,23fqKkggKUBHNkbKtXEls4,"[edm, pop, pop dance, tropical house]",8444136,75,Playlist 2
55,Harry Styles,6KImCVD70vtIoJWnq6nGn3,[pop],30346144,83,Playlist 2
29,James Blunt,7KMqksf0UMdyA0UCf4R3ux,[neo mellow],2900673,69,Playlist 1


In [13]:
a1 = dict(Counter(list(playlist1.explode('Artist(s)')['Artist(s)'])))
a2 = dict(Counter(list(playlist2.explode('Artist(s)')['Artist(s)'])))

In [14]:
tc = []
p1 = []
p2 = []
for a in list(all_artists['Name']):
    c1 = 0
    c2 = 0
    if a in list(a1.keys()):
        c1 = a1[a]
    if a in list(a2.keys()):
        c2 = a2[a]
    tc.append(c1+c2)
    p1.append(c1)
    p2.append(c2)

all_artists['Total Count'] = tc
all_artists['Count in Playlist 1'] = p1
all_artists['Count in Playlist 2'] = p2
all_artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2
59,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,"[canadian pop, pop]",74884572,87,Playlist 2,2,0,2
54,DJ Snake,540vIaP2JwjQb9dm3aArA4,"[dance pop, edm, electronic trap, pop, pop dance]",8779692,76,Playlist 2,1,0,1
39,Tainy,0GM7qgcRCORpGnfcN2tCiB,"[pop reggaeton, reggaeton, trap latino, urbano...",1842674,77,Playlist 2,1,0,1
74,Maren Morris,6WY7D3jk8zTrHtmkqqo5GI,"[classic texas country, contemporary country]",777453,68,Playlist 2,1,0,1
45,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,"[canadian pop, pop, viral pop]",42862332,80,Playlist 2,3,0,3


In [15]:
artist_presence = {}
for a, p in zip(list(all_artists['Name']), list(all_artists['Playlist'])):
    artist_presence[a] = p

In [18]:
artist_list = np.array(all_artists['Name'])
x = 0
lim = len(artist_list) // 10
y = x + lim
artist_dict = {}
print(x, y, lim)

0 8 8


In [19]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    x = y
    y = len(artist_list)
print(x, y, lim)

Nothing New (feat. Phoebe Bridgers) (Taylor’s Version) (From The Vault) https://open.spotify.com/track/01K4zKU104LyJ8gMb7227B 78
The Last Time (feat. Gary Lightbody of Snow Patrol) (Taylor’s Version) https://open.spotify.com/track/0y6kdSRCVQhSsHSpWvTUm7 73
ME! (feat. Brendon Urie of Panic! At The Disco) https://open.spotify.com/track/2Rk4JlNc2TPmZe2af99d45 80
Lover (Remix) [feat. Shawn Mendes] https://open.spotify.com/track/3i9UVldZOE0aD0JnyfAZZ0 77
You Need To Calm Down - Clean Bandit Remix https://open.spotify.com/track/7Bjf96uNizpvXLvNyOJpst 52
End Game https://open.spotify.com/track/2x0WlnmfG39ZuDmstl9xfX 82
You All Over Me (feat. Maren Morris) (Taylor’s Version) (From The Vault) https://open.spotify.com/track/4CHpVfAhuxNJ3ibExe6kxO 66
This Is The Time (feat. John Mayer) - Live at Shea Stadium, Queens, NY - July 2008 https://open.spotify.com/track/5Wf8sdQL7e9vHOVBDdGRdd 31
8 16 8


In [20]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

The Saints Are Coming https://open.spotify.com/track/6uQL0epVs9N9Bergafaf6x 44
You’re The Best Thing About Me - U2 Vs. Kygo https://open.spotify.com/track/4QWSk7Ynu34r3EOHSwp9nI 46
Runaway Horses (feat. Phoebe Bridgers) - Abridged https://open.spotify.com/track/3PeCWDczrGqdD1E3mbok2X 47
Caution - Clean Bandit Remix https://open.spotify.com/track/73A4FC5eIihO0swjveMDTl 34
16 24 8


In [21]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

BLOW (with Chris Stapleton & Bruno Mars) https://open.spotify.com/track/6EbduYTr62sSzvl36wWiFM 64
South of the Border (feat. Camila Cabello & Cardi B) https://open.spotify.com/track/4vUmTMuQqjdnvlZmAH61Qk 76
The Joker And The Queen (feat. Taylor Swift) https://open.spotify.com/track/6N1K5OVVCopBjGViHs2IvP 70
Pistol of Fire - Mark Ronson Remix https://open.spotify.com/track/2qOOsAgIpqXElQN7fkfj9N 23
24 32 8


In [22]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Payphone https://open.spotify.com/track/4P0osvTXoSYZZC2n8IFH3c 73
Button (feat. Anuel AA & Tainy) https://open.spotify.com/track/6FDKYM0lBf6zqrg8cFokPd 50
Beautiful Mistakes (feat. Megan Thee Stallion) https://open.spotify.com/track/5zFglKYiknIxks8geR8rcL 80
Let Somebody Go https://open.spotify.com/track/4cGqn0E8JCSY9gQllQj4Mf 75
My Universe - David Guetta Remix https://open.spotify.com/track/5FvxRvrQ8qzKjBJ6ST9aiu 53
Something Just Like This - Tokyo Remix https://open.spotify.com/track/5ffwnb6hqsKZp8cG6q90OW 51
My Universe https://open.spotify.com/track/3FeVmId7tL5YN8B7R3imoM 81
Let Somebody Go - Kygo Remix https://open.spotify.com/track/31JwHrEcDWkilfTkFZlKNr 52
Magnolia (feat. John Mayer) https://open.spotify.com/track/0VzLAYIns618H651sG09Gp 53
Give Me One Reason - Live https://open.spotify.com/track/6jTqomGlRFQ52EgIIWBfnY 42
32 40 8


In [23]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Elastic Heart - From "The Hunger Games: Catching Fire" Soundtrack https://open.spotify.com/track/6x0ex5urwhqB3AYLREVRUd 59
Floating Through Space https://open.spotify.com/track/7l339wfJlx6Ea2ye0xSDm5 0
Forever Yours - Avicii Tribute https://open.spotify.com/track/3o1CUVeHIid49sabk6A6Nf 62
Raging (feat. Kodaline) https://open.spotify.com/track/2Vhd5uZB4brKHmM0MNxkZo 64
It Ain’t Me (with Selena Gomez) https://open.spotify.com/track/12GEpg2XOPyqk03JZEZnJs 69
Lose Somebody https://open.spotify.com/track/1sgDyuLooyvEML4oHspNza 70
Like It Is https://open.spotify.com/track/3frUvGrmGcay91lvFbOgsN 61
ON (Feat. Sia) https://open.spotify.com/track/4JWbH50WabthGhmtduR8b9 58
Boy With Luv (feat. Halsey) https://open.spotify.com/track/4a9tbd947vo9K8Vti9JwcI 77
Butter - Megan Thee Stallion Remix https://open.spotify.com/track/474VqnPDr0251lsQBB8sQB 63
Make It Right (feat. Lauv) https://open.spotify.com/track/2WH4oy1SYppZrcdgKwSA0j 67
A Brand New Day (BTS World Original Soundtrack) (Pt. 2) https://open

In [24]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

FRIENDS https://open.spotify.com/track/08bNPGLD8AhKpnnERrAc6G 80
Happier https://open.spotify.com/track/7BqHUALzNBTanL6OvsqmC1 85
Be Kind (with Halsey) https://open.spotify.com/track/3Z8FwOEN59mRMxDCtb8N0A 75
Bad Bitches (feat. Megan Thee Stallion) https://open.spotify.com/track/2OYfxjLnQuPo7Co1WH0Z1S 43
We Don't Talk Anymore (feat. Selena Gomez) https://open.spotify.com/track/37FXw5QGFN7uwwsLy8uAc0 71
Attention - David Guetta Remix https://open.spotify.com/track/7ksmqYjUIkT7OFvvhw7Q7s 44
Left and Right (Feat. Jung Kook of BTS) https://open.spotify.com/track/5Odq8ohlgIbQKMZivbWkEo 82
Heaven - David Guetta & MORTEN Remix https://open.spotify.com/track/2087OWUqfOMjCUdX3JTxOg 0
Waiting For Love - Marshmello Remix https://open.spotify.com/track/4gnvnnm2oYy8IsUZ104wOS 38
Lost In Japan - Remix https://open.spotify.com/track/575NJxNUVDqwJGdzBrlLbv 61
Monster (Shawn Mendes & Justin Bieber) https://open.spotify.com/track/2Z8yfpFX0ZMavHkcIeHiO1 71
Summer of Love (Shawn Mendes & Tainy) https://op

In [25]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Rockabye (feat. Sean Paul & Anne-Marie) https://open.spotify.com/track/5knuzwU65gJK7IF5yJsuaW 76
Symphony (feat. Zara Larsson) https://open.spotify.com/track/72gv4zhNvRVdQA0eOenCal 80
New Eyes (feat. Lizzo) https://open.spotify.com/track/6xyJHp3nAJGOc2Xy7ACsg1 30
Uptown Funk (feat. Bruno Mars) https://open.spotify.com/track/32OlwWuMpZ6b0aN2RZOeMS 85
Find U Again (feat. Camila Cabello) https://open.spotify.com/track/1HpzOCZbNWzxvvXfSGtSrX 56
Nothing Breaks Like a Heart (feat. Miley Cyrus) https://open.spotify.com/track/02dRkCEc8Q5ch4TTcnLxOn 77
Sweetest Pie https://open.spotify.com/track/7mFj0LlWtEJaEigguaWqYh 73
Sweetest Pie - David Guetta Dance Remix https://open.spotify.com/track/5xUJYQA7f6WU77N8VDASBH 54
Middle https://open.spotify.com/track/0g5EKLgdKvNlln7TNqBByK 77
Let Me Love You - Zedd Remix https://open.spotify.com/track/7xufmkFUZoSjX1mxWupnKY 40
Let Me Love You https://open.spotify.com/track/0lYBSQXN6rCTvUZvg9S0lU 86
Fuego (with Sean Paul & Anitta, feat. Tainy) https://open.sp

In [26]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Physical (feat. Gwen Stefani) [Mark Ronson Remix] [Mixed] https://open.spotify.com/track/1GGuz6FiDdy6uH545iwYHn 45
Stay - Symphony Orchestra Version https://open.spotify.com/track/5SnanJRsBYK27l2hio8gOK 6
The Feeling https://open.spotify.com/track/7E390nZTMqEbrNC1TmHd42 62
Unstable (feat. The Kid LAROI) https://open.spotify.com/track/137mSruegm1u6x9NvwiagR 68
Overboard (Live) https://open.spotify.com/track/5h4Z1fyzlAZhGPRaQz7ima 45
fuck, i'm lonely https://open.spotify.com/track/2RNsweBBb1a6tAHXQtt2xH 68
Who https://open.spotify.com/track/0pSqYDTjY1Xt86usTQslAx 74
Prisoner (feat. Dua Lipa) https://open.spotify.com/track/2Oycxb8QbPkpHTo8ZrmG0B 79
Muddy Feet (feat. Sia) https://open.spotify.com/track/6QSR2OrZOICOXfFB5Wswt1 63
Happy Xmas (War Is Over) (feat. Sean Ono Lennon) https://open.spotify.com/track/5KHzV8Gf3aweIMImliFAOs 61
Sunshine https://open.spotify.com/track/6oNRhdGxT0CiypPQHyDiKW 41
2U (feat. Justin Bieber) https://open.spotify.com/track/61WR5FfGNLJxpLsAc61qWQ 61
Baby Don't H

In [27]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Everywhere (BBC Children In Need) https://open.spotify.com/track/0z0kUUd2jPQiI2Ob8VXajc 56
Seeing Blind https://open.spotify.com/track/65wnZsZLKJimfK1cTFOfa1 64
Derezzed - Remixed by Avicii https://open.spotify.com/track/1rn6JIHBuBZiwvA57jeoOB 41
Instant Crush (feat. Julian Casablancas) https://open.spotify.com/track/2cGxRwrMyEAp8dEbuZaVv6 77
Alone - Clean Bandit MFF Remix https://open.spotify.com/track/483v27PMHWBtPFyXQ3rElQ 30
Versace on the Floor (Bruno Mars vs. David Guetta) https://open.spotify.com/track/5UJshy5HDHdNEVqlJwzsQB 59
The Bones - with Hozier https://open.spotify.com/track/1yTTMcUhL7rtz08Dsgb7Qb 67
72 80 8


In [28]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Follow https://open.spotify.com/track/5aXTfpNOqLj35ydEiLio67 60
Scared to Be Lonely https://open.spotify.com/track/3ebXMykcMXOcLeJ9xZ17XH 80
So Far Away (feat. Jamie Scott & Romy Dya) https://open.spotify.com/track/0OlnLZY4cmQzT6ZGttvWBM 68
Torn Apart, Pt II (Bastille Vs. GRADES Vs. Lizzo) https://open.spotify.com/track/6Gy30w4ebJg02BjLdULdNm 40
Oh Na Na https://open.spotify.com/track/03NqHNdG3Ur6kciQWK6e8V 58
Bam Bam (feat. Ed Sheeran) https://open.spotify.com/track/0QBzMgT7NIeoCYy3sJCof1 78
Crown (with Camila Cabello & Grey) https://open.spotify.com/track/7qrMWXNwE1aD1o1sBnyIAh 50
Dusk Till Dawn (feat. Sia) - Radio Edit https://open.spotify.com/track/3e7sxremeOE3wTySiOhGiP 81
I Don’t Wanna Live Forever (Fifty Shades Darker) https://open.spotify.com/track/55n9yjI6qqXh5F2mYvUc2y 84
Die For You (with Ariana Grande) - Remix https://open.spotify.com/track/4W4fNrZYkobj539TOWsLO2 82
Starboy https://open.spotify.com/track/7MXVkk9YMctZqd1Srtv4MB 93
Can't Feel My Face - Martin Garrix Remix htt

In [29]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Marry The Night - Zedd Remix https://open.spotify.com/track/3acHQsIoaXtCPX96CDgivm 30
Marry The Night - The Weeknd & Illangelo Remix https://open.spotify.com/track/5bfivmKwgSni611w5gmAFa 28
Rain On Me (with Ariana Grande) https://open.spotify.com/track/7ju97lgwC2rKQ6wwsf9no9 74
Starstruck https://open.spotify.com/track/2DLsZCx5UEqVB3vY3kZWWB 54
On My Love https://open.spotify.com/track/0dxp7DyPJEo6JhtnjUtBnN 82
Good as Hell (feat. Ariana Grande) https://open.spotify.com/track/2rjjNUYONZqC7wSd1i6OFq 62
88 83 8


In [30]:
collabs = pd.concat(artist_dict.values(), ignore_index=True)

In [31]:
presence = []
for a1, a2 in zip(list(collabs['Artist 1']), list(collabs['Artist 2'])):
    if artist_presence[a1] == artist_presence[a2] and artist_presence[a1] != "Both":
        presence.append("Artists from Same Playlist")
    if artist_presence[a1] == artist_presence[a2] and artist_presence[a1] == "Both":
        presence.append("Both Artists in Same Playlists")
    if artist_presence[a1] != artist_presence[a2] :
        presence.append("Artists in Different Playlists")

collabs['Artist From'] = presence
collabs.sample(5)

,Artist 1,Artist 2,Popular Collab,URL,Popularity,Artist From
96,Wiz Khalifa,Charlie Puth,See You Again (feat. Charlie Puth),https://open.spotify.com/track/2JzZzZUQj3Qff7w...,85.0,Artists from Same Playlist
110,Halsey,Clean Bandit,Alone - Clean Bandit MFF Remix,https://open.spotify.com/track/483v27PMHWBtPFy...,30.0,Artists from Same Playlist
49,Avicii,David Guetta,Heaven - David Guetta & MORTEN Remix,https://open.spotify.com/track/2087OWUqfOMjCUd...,0.0,Artists from Same Playlist
128,Anne-Marie,Marshmello,Alarm - Marshmello Remix,https://open.spotify.com/track/2NvyKQtbrTYyiCH...,42.0,Artists from Same Playlist
47,Charlie Puth,David Guetta,Attention - David Guetta Remix,https://open.spotify.com/track/7ksmqYjUIkT7OFv...,44.0,Artists from Same Playlist


In [32]:
genre1 = dict(Counter(list(artist1.explode(column='Genres')['Genres'])))
#genre1 = {'Genre': list(genre1.keys()),
#          'Count': list(genre1.values())}
genre2 = dict(Counter(list(artist2.explode(column='Genres')['Genres'])))
#genre2 = {'Genre': list(genre2.keys()),
#          'Count': list(genre2.values())}

In [33]:
genre_list = []
genre_list.extend(list(genre1.keys()))
genre_list.extend(list(genre2.keys()))

genre_list = [x for x in set(genre_list)]

In [34]:
tc = []
cp1 = []
cp2 = []
pl = []

for g in genre_list:
    p1 = 0
    p2 = 0
    f = 0
    if g in genre1.keys():
        p1 = genre1[g]
    if g in genre2.keys():
        p2 = genre2[g]
    tc.append(p1+p2)
    cp1.append(p1)
    cp2.append(p2)
    if p1 == 0 and p2 != 0:
        pl.append("Playlist 2")
    elif p1 != 0 and p2 == 0:
        pl.append("Playlist 1")
    else:
        pl.append("Both")

all_genres = pd.DataFrame({
    'Genre': genre_list,
    'Playlist': pl,
    'Total Count': tc,
    'Count in Playlist 1': cp1,
    'Count in Playlist 2': cp2
})

all_genres.sample(5)

,Genre,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2
110,lilith,Playlist 1,1,1,0
31,singer-songwriter,Playlist 1,4,4,0
38,baroque pop,Playlist 1,1,1,0
84,celtic rock,Playlist 1,1,1,0
23,britpop,Playlist 1,1,1,0


In [35]:
cg = list(all_genres[all_genres['Playlist'] == "Both"]['Genre'])
common_genres = []
for g in cg:
    g = g.split()[0]
    common_genres.append(g)

common_artists = list(all_artists[all_artists['Playlist'] == "Both"]['Name'])
common_artists_id = list(all_artists[all_artists['Playlist'] == "Both"]['ID'])

In [36]:
common_genre_top_artists = pd.DataFrame()
for g1, g2 in zip(common_genres, cg):
    an = []
    aid = []
    ag = []
    ap = []
    #print(g2)
    genre_artists = get_top_artists_for_genre(g1, 25)
    for a in genre_artists:
        gc = a['genres']
        if g2 in gc and a['name'] not in common_artists:
            an.append(a['name'])
            aid.append(a['id'])
            ap.append(a['popularity'])
            ag.append(g2)
    ga = pd.DataFrame({
        'Name': an,
        'ID': aid,
        'Genre': ag,
        'Popularity': ap
    }).sort_values(by='Popularity', ascending=False, ignore_index=True)
    if len(ga) > 5:
        ga = ga.head(5)
    #print(ga)
    common_genre_top_artists = pd.concat([common_genre_top_artists, ga])
    #print()
common_genre_top_artists.sample(5)

,Name,ID,Genre,Popularity
4,blink-182,6FBDaR13swtiWwGhX1WQsP,modern rock,78.0
0,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,pop,92.0
1,JVKE,164Uj4eKjl6zTBKfJLFKKK,singer-songwriter pop,75.0
1,Wham!,5lpH0xAS4fVfLkACg9DAuM,new wave,73.0
4,Lykke Li,6oBm8HB0yfrIc9IHbxs6in,swedish pop,66.0


In [37]:
scg = []
songs_common_genres = pd.DataFrame()
for a, g, i in zip(list(common_genre_top_artists['Name']), list(common_genre_top_artists['Genre']), list(common_genre_top_artists['ID'])):
    #print(a)
    common_genre_songs = {
        'Track_Name': [],
        'Track_ID': [],
        'URL': [],
        'Artist(s)': [],
        'Album': [],
        'Popularity': [],
        'Duration': [],
        'Tempo': [],
        'Key': [],
        'Mode': [],
        'Danceability': [],
        'Energy': [],
        'Loudness': [],
        'Speechiness': [],
        'Acousticness': [],
        'Instrumentalness': [],
        'Liveliness': [],
        'Valence': [],
        'Time_Signature': []
    }
    songs = get_top_tracks_for_artist(i, num_tracks=25)
    for s in songs:
        #print(s['name'], [artist['name'] for artist in s['artists']])
        if s['uri'] not in list(playlists['Track_ID']):
            #print(s['name'], [artist['name'] for artist in s['artists']])
            #print()
            f = get_track_mood(s['uri'])
            common_genre_songs['Track_Name'].append(s['name'])
            common_genre_songs['Track_ID'].append(s['uri'])
            common_genre_songs['URL'].append(s['external_urls']['spotify'])
            common_genre_songs['Artist(s)'].append([artist['name'] for artist in s['artists']])
            common_genre_songs['Album'].append(s['album']['name'])
            common_genre_songs['Popularity'].append(s['popularity'])
            common_genre_songs['Duration'].append(round(f[0]['duration_ms'] / 60000, 2))
            common_genre_songs['Tempo'].append(f[0]['tempo'])
            common_genre_songs['Key'].append(f[0]['key'])
            common_genre_songs['Mode'].append(f[0]['mode'])
            common_genre_songs['Danceability'].append(f[0]['danceability'])
            common_genre_songs['Energy'].append(f[0]['energy'])
            common_genre_songs['Loudness'].append(f[0]['loudness'])
            common_genre_songs['Speechiness'].append(f[0]['speechiness'])
            common_genre_songs['Acousticness'].append(f[0]['acousticness'])
            common_genre_songs['Instrumentalness'].append(f[0]['instrumentalness'])
            common_genre_songs['Liveliness'].append(f[0]['liveness'])
            common_genre_songs['Valence'].append(f[0]['valence'])
            common_genre_songs['Time_Signature'].append(f[0]['time_signature'])
    common_genre_songs = pd.DataFrame(common_genre_songs).sort_values(by='Popularity', ascending=False, ignore_index=True).head(5)
    #print(common_genre_songs)
    #print()
    songs_common_genres = pd.concat([songs_common_genres, common_genre_songs], ignore_index=True)
    scg.extend([g]*len(common_genre_songs))
songs_common_genres.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature
241,Why'd You Only Call Me When You're High?,spotify:track:086myS9r57YsLbJpU0TgK9,https://open.spotify.com/track/086myS9r57YsLbJ...,[Arctic Monkeys],AM,90,2.69,92.004,2,1,0.691,0.631,-6.478,0.0368,0.0483,0.000011,0.1040,0.800,4
105,I'm Still Standing,spotify:track:1jDJFeK9x3OZboIAHsY9k2,https://open.spotify.com/track/1jDJFeK9x3OZboI...,[Elton John],Too Low For Zero,86,3.06,176.808,6,1,0.504,0.904,-6.863,0.1790,0.3560,0.121000,0.1400,0.772,4
326,How Do You Do!,spotify:track:4Hn14NabX651eOOzsLL8Uc,https://open.spotify.com/track/4Hn14NabX651eOO...,[Roxette],Tourism (Extended Version),66,3.15,120.874,0,1,0.619,0.720,-6.584,0.0275,0.0194,0.000002,0.2180,0.852,4
21,What's My Age Again?,spotify:track:4LJhJ6DQS7NwE7UKtvcM52,https://open.spotify.com/track/4LJhJ6DQS7NwE7U...,[blink-182],Enema Of The State,81,2.47,157.660,6,1,0.391,0.951,-7.419,0.0692,0.0126,0.000378,0.0878,0.468,4
188,We Are The People - southstar Remix,spotify:track:3vLMN02wGnPBdjRVvewedS,https://open.spotify.com/track/3vLMN02wGnPBdjR...,"[Empire of the Sun, southstar]",We Are The People (southstar Remix),65,3.93,150.014,7,1,0.717,0.832,-6.404,0.0396,0.0018,0.033400,0.1300,0.128,4


In [38]:
ca = []
songs_common_artists = pd.DataFrame()
for a, i in zip(common_artists, common_artists_id):
    common_artist_songs = {
        'Track_Name': [],
        'Track_ID': [],
        'URL': [],
        'Artist(s)': [],
        'Album': [],
        'Popularity': [],
        'Duration': [],
        'Tempo': [],
        'Key': [],
        'Mode': [],
        'Danceability': [],
        'Energy': [],
        'Loudness': [],
        'Speechiness': [],
        'Acousticness': [],
        'Instrumentalness': [],
        'Liveliness': [],
        'Valence': [],
        'Time_Signature': []
    }
    songs = get_top_tracks_for_artist(i, num_tracks=25)
    for s in songs:
        if s['uri'] not in list(playlists['Track_ID']):
            f = get_track_mood(s['uri'])
            common_artist_songs['Track_Name'].append(s['name'])
            common_artist_songs['Track_ID'].append(s['uri'])
            common_artist_songs['URL'].append(s['external_urls']['spotify'])
            common_artist_songs['Artist(s)'].append([artist['name'] for artist in s['artists']])
            common_artist_songs['Album'].append(s['album']['name'])
            common_artist_songs['Popularity'].append(s['popularity'])
            common_artist_songs['Duration'].append(round(f[0]['duration_ms'] / 60000, 2))
            common_artist_songs['Tempo'].append(f[0]['tempo'])
            common_artist_songs['Key'].append(f[0]['key'])
            common_artist_songs['Mode'].append(f[0]['mode'])
            common_artist_songs['Danceability'].append(f[0]['danceability'])
            common_artist_songs['Energy'].append(f[0]['energy'])
            common_artist_songs['Loudness'].append(f[0]['loudness'])
            common_artist_songs['Speechiness'].append(f[0]['speechiness'])
            common_artist_songs['Acousticness'].append(f[0]['acousticness'])
            common_artist_songs['Instrumentalness'].append(f[0]['instrumentalness'])
            common_artist_songs['Liveliness'].append(f[0]['liveness'])
            common_artist_songs['Valence'].append(f[0]['valence'])
            common_artist_songs['Time_Signature'].append(f[0]['time_signature'])
    common_artist_songs = pd.DataFrame(common_artist_songs).sort_values(by='Popularity', ascending=False, ignore_index=True).head(5)
    songs_common_artists = pd.concat([songs_common_artists, common_artist_songs], ignore_index=True)
    ca.extend([a]*len(common_artist_songs))
songs_common_artists.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature
30,Just like Heaven,spotify:track:76GlO5H5RT6g7y0gev86Nk,https://open.spotify.com/track/76GlO5H5RT6g7y0...,[The Cure],"Kiss Me, Kiss Me, Kiss Me",71,3.53,151.219,9,1,0.430,0.971,-3.845,0.0524,0.0271,0.000022,0.1850,0.764,4
23,Sugar,spotify:track:2iuZJX9X9P0GKaE93xcPjk,https://open.spotify.com/track/2iuZJX9X9P0GKaE...,[Maroon 5],V,83,3.92,120.076,1,1,0.748,0.788,-7.055,0.0334,0.0591,0.000000,0.0863,0.884,4
11,Shape of You,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://open.spotify.com/track/7qiZfU4dY1lWllz...,[Ed Sheeran],÷ (Deluxe),88,3.90,95.977,1,0,0.825,0.652,-3.183,0.0802,0.5810,0.000000,0.0931,0.931,4
15,RUNAWAY,spotify:track:07fbDnkKdZGk1gLvknxrns,https://open.spotify.com/track/07fbDnkKdZGk1gL...,[OneRepublic],RUNAWAY,81,2.39,163.033,3,0,0.624,0.684,-5.507,0.0513,0.0336,0.000000,0.0771,0.750,4
0,Is It Over Now? (Taylor's Version) (From The V...,spotify:track:1Iq8oo9XkmmvCQiGOfORiz,https://open.spotify.com/track/1Iq8oo9XkmmvCQi...,[Taylor Swift],1989 (Taylor's Version),94,3.82,100.012,0,1,0.596,0.658,-7.346,0.0360,0.0504,0.000000,0.1270,0.176,4


In [39]:
new_playlist = pd.concat([songs_common_genres, songs_common_artists], ignore_index=True).sort_values(by='Tempo', ignore_index=True)
new_playlist.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature
77,Good Life,spotify:track:6OtCIsQZ64Vs1EbzztvAv4,https://open.spotify.com/track/6OtCIsQZ64Vs1Eb...,[OneRepublic],Waking Up,76,4.22,94.988,6,1,0.634,0.690,-7.804,0.0520,0.07710,0.000000,0.1320,0.6450,4
262,Ribs,spotify:track:2MvvoeRt8NcOXWESkxWn3g,https://open.spotify.com/track/2MvvoeRt8NcOXWE...,[Lorde],Pure Heroine,81,4.32,127.978,4,1,0.511,0.472,-9.277,0.0910,0.53400,0.614000,0.1100,0.0399,4
88,All The Stars (with SZA),spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,https://open.spotify.com/track/3GCdLUSnKSMJhs4...,"[Kendrick Lamar, SZA]",Black Panther The Album Music From And Inspire...,86,3.87,96.924,8,1,0.698,0.633,-4.946,0.0597,0.06050,0.000194,0.0926,0.5520,4
62,Can't Stop,spotify:track:3ZOEytgrvLwQaqXreDs2Jx,https://open.spotify.com/track/3ZOEytgrvLwQaqX...,[Red Hot Chili Peppers],By the Way (Deluxe Edition),86,4.48,91.455,9,1,0.618,0.938,-3.442,0.0456,0.01790,0.000000,0.1670,0.8750,4
209,Don't Say Goodbye (feat. Tove Lo),spotify:track:0GTPmh5G40joBg93vcSQNq,https://open.spotify.com/track/0GTPmh5G40joBg9...,"[Alok, Ilkay Sencan, Tove Lo]",Don't Say Goodbye (feat. Tove Lo),71,3.11,120.062,0,0,0.698,0.719,-4.923,0.0455,0.00829,0.000004,0.3260,0.3640,4


In [40]:
songs_common_genres['Common Genre'] = scg
songs_common_genres.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Genre
85,Upside Down,spotify:track:0aJrYfARfCKzv6gXjhs4SZ,https://open.spotify.com/track/0aJrYfARfCKzv6g...,[Jack Johnson],Upside Down,76,3.49,102.477,4,1,0.792,0.647,-8.314,0.0450,0.2550,0.000223,0.1320,0.694,4,neo mellow
84,Gravity,spotify:track:3SktMqZmo3M9zbB7oKMIF7,https://open.spotify.com/track/3SktMqZmo3M9zbB...,[John Mayer],Continuum,74,4.10,123.799,7,1,0.753,0.326,-9.883,0.0342,0.7310,0.001310,0.2400,0.237,3,neo mellow
254,One Step Closer,spotify:track:3K4HG9evC7dg3N0R9cYqk4,https://open.spotify.com/track/3K4HG9evC7dg3N0...,[Linkin Park],Hybrid Theory (Bonus Edition),81,2.62,95.136,9,1,0.492,0.969,-4.419,0.0491,0.0014,0.000001,0.0787,0.538,4,rock
143,Desire (with Sam Smith),spotify:track:22dUzMFttcR3uU17NcOAIv,https://open.spotify.com/track/22dUzMFttcR3uU1...,"[Calvin Harris, Sam Smith]",Desire (with Sam Smith),86,2.99,140.009,11,0,0.587,0.924,-5.433,0.0457,0.0311,0.017100,0.2300,0.509,4,uk pop
178,Pressure (feat. Tove Lo),spotify:track:4ga4WvRNhMsIL38pkBbnIN,https://open.spotify.com/track/4ga4WvRNhMsIL38...,"[Martin Garrix, Tove Lo]",Pressure (feat. Tove Lo),66,2.41,114.978,7,0,0.859,0.558,-8.991,0.0902,0.6590,0.324000,0.1240,0.357,4,metropopolis


In [41]:
songs_common_artists['Common Artist'] = ca
songs_common_artists.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Artist
26,Viva La Vida,spotify:track:1mea3bSkSGXuIRvnydlB5b,https://open.spotify.com/track/1mea3bSkSGXuIRv...,[Coldplay],Viva La Vida or Death and All His Friends,88,4.04,138.015,5,0,0.486,0.617,-7.115,0.0287,0.095400,0.000003,0.1090,0.417,4,Coldplay
16,Sunshine,spotify:track:5r43qanLhUCdBj8HN3fa6B,https://open.spotify.com/track/5r43qanLhUCdBj8...,[OneRepublic],Sunshine,80,2.73,140.069,5,1,0.697,0.875,-4.621,0.0340,0.137000,0.000000,0.2800,0.774,4,OneRepublic
4,"""Slut!"" (Taylor's Version) (From The Vault)",spotify:track:71BqAINEnezjQfxE4VuJfq,https://open.spotify.com/track/71BqAINEnezjQfx...,[Taylor Swift],1989 (Taylor's Version),91,3.01,77.983,7,1,0.629,0.413,-13.178,0.0550,0.345000,0.000076,0.0603,0.306,4,Taylor Swift
31,Friday I'm in Love,spotify:track:263aNAQCeFSWipk896byo6,https://open.spotify.com/track/263aNAQCeFSWipk...,[The Cure],Wish,70,3.57,136.203,0,0,0.526,0.762,-12.437,0.0342,0.000972,0.000116,0.3380,0.505,4,The Cure
29,A Sky Full of Stars,spotify:track:0FDzzruyVECATHXKHFs9eJ,https://open.spotify.com/track/0FDzzruyVECATHX...,[Coldplay],Ghost Stories,87,4.46,124.970,6,1,0.545,0.675,-6.474,0.0279,0.006170,0.001970,0.2090,0.162,4,Coldplay


In [42]:
playlist1.to_csv("Playlist1.csv")
playlist2.to_csv("Playlist2.csv")

playlists.to_csv("Combined.csv")

artist1.to_csv("Artists1.csv")
artist2.to_csv("Artists2.csv")

collabs.to_csv("All Collaborations.csv", index=False)

all_artists.to_csv("All Artists.csv", index=False)

all_genres.to_csv("Genres.csv", index=False)

songs_common_genres.to_csv("Common Genre Songs.csv", index=False)
songs_common_artists.to_csv("Common Artist Songs.csv", index=False)
new_playlist.to_csv("New Playlist.csv", index=False)